In [121]:


import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime as dt


from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve, confusion_matrix, mean_squared_error, accuracy_score, precision_score, brier_score_loss
from sklearn.metrics import auc, roc_auc_score, roc_curve, recall_score, f1_score, log_loss, average_precision_score


from statistics import mode
from sklearn import preprocessing
from pandas import Series
from sklearn.model_selection import GridSearchCV
import pandas_profiling
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/output'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [122]:
RANDOM_SEED=42

In [123]:
train_data = pd.read_csv("/kaggle/input/sf-dst-scoring/train.csv")
test_data = pd.read_csv("/kaggle/input/sf-dst-scoring/test.csv")
sample=pd.read_csv('/kaggle/input/sf-dst-scoring/sample_submission.csv')
test_data.info()

In [124]:
#Функция для рассчета стат.величин и выбросов
#Для строковых столбцов
def plot_str(df,col):

    print('Распределение для столбца (не числовой):', col)
    fig,ax=plt.subplots()
    sns.countplot(x=df.loc[:,col], ax=ax)
    plt.show()
#поиск пустых  Nan значений в символьном  столбце, расчет процента потерянных значений
    n=100-(df[col].count()/df.shape[0]*100)
    print('уникальных значений ', len(df[col].dropna().unique()))
    print ('пустых значений,%', round(n,2))
    df.default.unique()

#Для числовых столбцов
def plot_num (df,col_name, borders=None):
    sns.set(rc={'figure.figsize':(5, 5)})
    print('Распределение для столбца (числовой):', col_name)
    
    fig,ax=plt.subplots()
    df.loc[:,col_name].hist(ax=ax)
    ax.set_xlabel(col_name)
    ax.set_ylabel('кол-во')
    plt.show()
    print (df[col_name].count())
    print (df.shape[0])

    n=100-(df[col_name].count()/df.shape[0]*100)
    print ('пустых значений,%', round(n,2))
    
    if borders is not None:
        print ('выбросы по условию ', borders)
        display(df[(~df.loc[:,col_name].between(borders[0],borders[1]))&pd.notnull(df.loc[:,col_name])])

#функция для рисования "усов"  
def get_boxplot(df,column):
    fig, ax = plt.subplots(figsize = (5, 5))
    sns.boxplot(x='default', y=column, 
                data=df.loc[df.loc[:, column].isin(df.loc[:, column].value_counts().index[:30])],
               ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot для ' + column)
    plt.show()   
    
#Функция по расчету коэффициента стьюдента из 
def get_stat_dif(df,column):
    cols = df.loc[:, column].value_counts(dropna =True).index[:30]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'score'], 
                        df.loc[df.loc[:, column] == comb[1], 'score']).pvalue <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break
#построение ROC AUC кривой 
def roc(y_test, y_probs):
    sns.set(rc={'figure.figsize':(5, 5)})

    fpr, tpr, threshold = roc_curve(y_test, y_probs)
    roc_auc = roc_auc_score(y_test, y_probs)
    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    plt.title('Logistic Regression ROC AUC = %0.5f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()

# построение confusion matrix    
def conf_matrix(lastmodel,X_test,y_test):
    sns.set(rc={'figure.figsize':(5, 5)})
    
    class_names = ['NonDefault', 'Default']
    titles_options = [("Confusion matrix", None)]
    for title, normalize in titles_options:
        disp = plot_confusion_matrix(lastmodel, X_test, y_test, 
                                     display_labels=class_names, 
                                     cmap=plt.cm.Blues, 
                                     normalize=normalize)
        disp.ax_.set_title(title)

        print(title)
        print(disp.confusion_matrix)

    plt.show()
#Метрики
def metrics(Y_valid, Y_pred):
    print('accuracy_score: {}'.format(np.round(accuracy_score(Y_valid, Y_pred), 4)))
    print('f1_score: {}'.format(np.round(f1_score(Y_valid, Y_pred), 4)))
    print('precision_score: {}'.format(np.round(precision_score(Y_valid, Y_pred), 4)))
    print('recall_score: {}'.format(np.round(recall_score(Y_valid, Y_pred), 4)))
    
# Самые значимые признаки
def impotant_cols(df,cols, data):
  imp_num = pd.Series(f_classif(df[df['test_file'] == 0][cols], df[df['test_file'] == 0]['default'])[0], index = cols)
  imp_num.sort_values(inplace = True)
  imp_num.plot(kind = 'barh', figsize=(15,10))
  return ''    

#Процедура для подготовки датасетов для обучения и валидации
def split_train_valid (union_df, train_label, test_file_label, target):
    df=union_df.copy()
    X_train=df[df[train_label]==1].drop (columns=[target,train_label,test_file_label,'client_id'])
    Y_train=df[df[train_label]==1][target] 
    
    X_valid=union_df[(union_df[train_label]==0) & (union_df[test_file_label]==0)].drop (columns=[target,train_label,test_file_label,'client_id']) 
    Y_valid=union_df[(union_df[train_label]==0) & (union_df[test_file_label]==0)][target]
    
    return X_train, X_valid, Y_train, Y_valid #, test_file


Посмотрим на Данные, пропуски и целевую переменную.

In [125]:
#Грфическое изображение пропусков данных
sns.set(rc={'figure.figsize':(10, 3)})
cols = train_data.columns[2:33] # 30 колонок

# желтый - пропущенные данные, синий - не пропущенные
colours = ['#000099', '#ffff00'] 
sns.heatmap(data=train_data[cols].notnull(), cmap=sns.color_palette(colours))
plt.show()
print ('Распределение целевой переменной')
sns.set(rc={'figure.figsize':(5, 5)})
print(train_data['default'].value_counts(normalize=True))

#гистограмма целевая переменная
fig,ax=plt.subplots()
sns.countplot(x=train_data['default'], ax=ax)
plt.show()

print ('Распределение целевой переменной с незаполненной графой "образование"')
fig,ax=plt.subplots()
print(train_data[train_data.education.isnull()]['default'].value_counts(normalize=True))
sns.countplot(x=train_data[train_data.education.isnull()]['default'], ax=ax)
plt.show()

train_data.head()

Исходя из описания датасета и информации о нем:
1. Практически нет пропусков, только по колонке education, позже заполним чаще всего встречающимся знаением или новым значением (определимся по результатам моделирования).
2. 19 колонок: 
    7 Числовых (client_id, age, decline_app_cnt, bki_request_cnt, income,score_bki,region_rating), 
    6 категориальных (education, home_address,work_address, sna, app_date, first_time) и 
    6 булевых (sex, car, car_type, good_work, foreign_passport, default)
3. Есть одна колонка с датами, которые подлежат преобразованию и генерации дополнительных признаков.
4. Целевая переменная  - default пропусков не содержит, но очень плохо сбалансирована: оценка default=1 встречается намного реже, это естественно, кредиты чаще дают, чем отказывают в выдаче кредита.
   В связи с этим нужно будет подумать, как сбалансирвоать обучающую выборку.

In [126]:
#Определяем списки числовых, категориальных и булевых столбцов
num_cols=['client_id', 'age', 'decline_app_cnt', 'bki_request_cnt', 'income','score_bki']
cat_cols=['education', 'home_address','work_address', 'sna', 'app_date', 'first_time','region_rating']
bool_cols=['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

## Посмотрим на данные:
### 1. Числовые столбцы

In [127]:
fig, axes = plt.subplots(1, 6, figsize=(25,5))
for col, i in zip(num_cols, range(6)):
    sns.histplot(train_data[col], kde=False, ax=axes.flat[i])
plt.show()    

#Строим boxplot для числовых столбцов


for column in num_cols:
    print (column)
    get_boxplot(train_data,column)



### Предварительные выводы по числовым столбцам. 
1. client_id - идентификатор клиента. Скорее всего кореллирует с датой подачи заявки и означет просто номер по порядку появления клиента в банке. Скорее всего для модели не пригодится, проверим позже.
2. age - возраст клиентов. Видно, что больше клиентов до 40-45 лет лет. 
3. decline_app_cnt, bki_request_cnt количество отказов и количество заявок - распределены неравномерно, для полуения более красивого распределения можно попробовать логарифмировать. Также имеются выбросы, попробуем заменить выбросы на верхнюю границу 4 квартиля либо на границу выброса, проверим результат при моделировании и валидации модели.
4. income - доход также распределен неравномерно, можно пробовать логарифмировать. Также можно поделить доход по категориям.
5. score_bki - скоринговый балл по БКИ - распределение приближено к нормальному
6. region_rating - рейтинг региона. На основании рейтинга создадим новый признак: относительная зарплата. Это средняя зарплата по сгруппированным по рейтингу регионам/зарплату. Признак логичный и должен дать некоторое увеличение качества классификации. 

### Распределение по нечисловым  столбцам

In [128]:
for column in train_data.columns:
    if (column in cat_cols or column in bool_cols)  and column not in ['index', 'app_date', 'default']:    
        plot_str(train_data,column)

### Предварительные выводы
1. app_date - преобразуем и сгенерируем новые признаки: 1. ДЕНЬ НЕДЕЛИ, 2. МЕСЯЦ 3.ДАВНОСТЬ ЗАЯВКМ (ОТ ТЕКУЩЕЙ ДАТЫ). Сам по себе признак потом удалить.
2. education - сначала заполним пропуски наиболее часто встречающимся значением - SCH, затем создадим новый вид признака для пустых значений, проверим что лучше повлияет на модель. также закодируем символные значения числовым кодом. Закодируем признак числами по порядку возрастания уровня образования. 
3. sex - также закодируем пол заемщика как 0 и 1. Мужчин и женщин примерно одинаковое количество среди заемщиков.
4. car, car_type - закодировать как 0 и 1. 
5. good_work - признак хорошей работы. также кодируем как 0 и 1
6. home_address, work_address - категория рабочего и домашнего адреса. Возможно будет кореллировать друг с другом и рейтингом региона. Нужно проверить.
7. first_time - давность наличия информации о заемщике.
8. foreign_passport - наличие загранпаспорта.


### Создаем единый датасет из train  и test

In [129]:
#Объединяем test и train для преобразования
#дополняем тест колонкой default, которая в нем отсутствует
test_data['default'] = 0
# test_file=1 - данные, которые уйдут в конечную тестовую выборку
# test_file=0 - данные, которые потом поделим на обучающию и валидирующую выборки

train_data['test_file'] = 0
test_data['test_file'] = 1
test_data['train'] = 0
Xt = train_data

# Делим train на тренировочную и валидационную
X_train, X_valid=train_test_split(Xt, train_size=0.7, random_state=42, stratify=Xt.default)


X_train['train']=1
X_valid['train']=0
train_data=X_train.append(X_valid, sort=False)
union_data = test_data.append(train_data, sort=False)
union_data.train=union_data.train.fillna(0)

### Преобразование категориальных переменных

In [130]:
#Преборазуем дату
union_data['app_date_date'] = pd.to_datetime(union_data['app_date'])
#генерируем признак - день недели
union_data['date_dow']= union_data['app_date_date'].apply(lambda x: dt.weekday(x))
#генерируем признак - месяц, год
union_data['date_month']= union_data['app_date_date'].map(lambda x: x.month)
union_data['date_year']= union_data['app_date_date'].map(lambda x: x.year)
#генерируем признак - давность заявки
union_data['date_age']= union_data['app_date_date'].apply(lambda x: (dt.today()-x).days )
#Удаляем ненужный столбец
union_data.drop('app_date', axis=1, inplace=True)


In [131]:
union_data[union_data['train']==1]

#### Смотрим, как распределяется количество отказов по дням недели, годам и месяцам:
В обучающей выборке увидим только данные за 2014 год, за 4 первых месяца. Использовать в обучающей выборке год и месяц не имеет смысла, посколку выборка по месяцам неполная (не охватывает год).

День недели заслуживает внимания, распределение построено логично: больше заявок рассматривается в будни. 

Построим таже boxplot признака "День недели": Отказ или подтверждение заявки по кредиту мало зависит от дня недели.

Проверим, зависит ли отказ по кредиту от давности заявки: построим boxplot для этого признака. По графику видно, что зависимость есть. Возможно, данный признак позволит улучшить модель.

Из признаков, полученных из даты, оставим только день недели и признак "давности" заявки.     

In [132]:
print(union_data[union_data['test_file']==0].date_year.value_counts()) 
print(union_data[union_data['test_file']==0].date_month.value_counts() )
print(union_data[union_data['test_file']==0].date_dow.value_counts()) 
fig,ax=plt.subplots()
sns.countplot(union_data[union_data['test_file']==0]['date_dow'], ax=ax)
plt.show()
fig, axes = plt.subplots(figsize=(10,5))
sns.boxplot(x=union_data[union_data['test_file']==0]['default'], y=union_data['date_dow'])
plt.show()
fig, axes = plt.subplots(figsize=(10,5))
sns.boxplot(x=union_data[union_data['test_file']==0]['default'], y=union_data['date_age'])


Превратим бинарные категориальные переменные в числовые, закодировав признаки. Также зашифруем столбец education и заполним пустые значения наиболее часто встречающимся значением.
В дальнейшем эти пустые значения следует проанализировать и, учитывая несбалансированность целевой переменной, проверить баланс целевой переменной и на этих пустых значениях. Возможно, следует для пустых значений сделать отдельный код.

По заполнению education: Выяснена градация уровней образования, поэтому кодируем признак в соотвтсвии с уровнями:
* 1 - SCH - school - те, у кого среднее образование (только школа).
* 2 - UGR - undergraduate - бакалавры.
* 3 - GRD - магистры
* 4 - PGR - postgraduate - учёная степень PhD (кандидаты наук по-нашему)
* 5 - ACD - высший уровень. Можно считать высшей категорией.
Таким образом, если упорядочить по возрастанию уровня образования, получим следующий список:

SCH, UGR, GRD, PGR, ACD


In [133]:
print(union_data.education.unique())
dict_ed = {'GRD':1, 'SCH':2, 'UGR':3, 'PGR':4, 'ACD':5}
dict_ed
union_data['education_cod'] = union_data['education'].map(dict_ed)
union_data.head()

Заполняем пропуски другим (отличным от имеющихся) видом образования, поскольку данный вид заполнения повлиял на результат в лучшую сторону (дал увеличение показателя ROC).

In [134]:
# Пропущенные значения - новый вид категоирального признака
union_data['education_cod']=union_data['education_cod'].fillna(6)
#Удаляем ненужный столбец
union_data.drop('education', axis=1, inplace=True)

In [135]:
#Кодируем бинарные признаки:
label_encoder = LabelEncoder()
for c in bool_cols:
    union_data[c] = label_encoder.fit_transform(union_data[c])
     
union_data.head()

In [136]:
#Удаляем лишние столбцы и строим первую "наивную" модель
union_data.drop(['app_date_date', 'date_month', 'date_year'], axis=1, inplace=True)


In [137]:

X_train, X_valid, Y_train, Y_valid=split_train_valid (union_data, 'train', 'test_file', 'default')
model_0 = LogisticRegression(solver='liblinear')
model_0.fit(X_train, Y_train)
Y_pred = model_0.predict(X_valid)

# Строим ROC AUC 

probs = model_0.predict_proba(X_valid)
probs = probs[:,1]
roc(Y_valid,probs)

# Строим матрицу ошибок

conf_matrix(model_0,X_valid,Y_valid)

# метрики
metrics(Y_valid, Y_pred)

### Результат неудовлетворительный: 
Несмотря на то, что кривая ROC AUC выглядит неплохо, другие показатели (точность, полнота, F-мера) очень низкие. Accuracy не используем из-за явного дисбаланса классов. По видимому, из-за дизбаланса обучающей выборки в сторону положительных оценок модель дает больше позитивных оценок, в связи с чем recall очень низкий.
План:
1. Построить корелляционную матрицу, определить лишние переменные.

2. Сгенерировать dummy-переменные, оценить результат.
3. Сбалансировать выборку по целевой переменной, оценить результат
4. Настроить гиперпараметры.

In [138]:
plt.rcParams['figure.figsize'] = (15,10)

matrix = np.triu(union_data[union_data['test_file'] == 0].corr())
sns.heatmap(union_data.corr(), annot=False, mask=matrix, cmap= 'coolwarm')

**1. Строим матрицу корреляции**

Сильная корреляция между признаками: 
car_type и car, 
work address и home_address

Собираем эти признаки как сумму:

также удаляем день недели как незначимый. 


In [139]:
impotant_cols(union_data,union_data[union_data['test_file'] == 0].columns, union_data.default)

Большое количетво выбросов по колонкам decline_app_cnt и bki_reques_cnt.
Попытка заменить выбросы в обучающей выборке на верхнюю границу 4 квартиля успехом не увенчалась, улучшения качества модели после замены выбросов не получилось. 

In [140]:
def quant_4 (data, col): #quant=0.25 или 0.75
   # print (data,col)
    IQR = data[col].quantile(q=0.75) - data[col].quantile(0.25)
    print ('IQR ',IQR)
    perc75 = data[col].quantile(0.75)
    
    high_border = perc75 + 1.5*IQR
    print ('high_border ',round(high_border,0))
    data[col] = data[col].apply(lambda x: high_border if x>high_border else x)
    return data

#quant_4(union_data,'decline_app_cnt')
#замена выбросов по количеству отклоненных заявок сильно ухудшило результат классификации.

#quant_4(union_data,'bki_request_cnt')
#quant_4(union_data,'income')
union_data.info()

### Добавление новых количественных признаков:
1. Разделим доход на группы по категориям: 0: <20000, 1: от 20 до 40т, 2: от 40 до 60т, 3: от 60-80т, 4: >80
2. Посчитаем средний доход по возрасту.

In [141]:
union_data['income_cat']=union_data['income'].copy()
#Категория дохода
union_data['income_cat']=union_data['income_cat'].apply(lambda x: 0 if x<=20000 else (1 if 20<x<=40 else (2 if 40<x<=60 else (3 if 60<x<=80 else 4))) )
mean_income=pd.DataFrame(union_data.groupby('region_rating')['income'].mean())
#Доход относительно среднего по категории региона
mean_income.columns=['income_y']
union_data=union_data.merge(mean_income, how='left', left_on='region_rating',  right_on='region_rating' )
union_data['income_reg']=union_data['income']/union_data['income_y']
union_data=union_data.drop(columns=['income_y'])

### Объединяем сильно коррелирующие признаки (собираем их попарно)

In [142]:
union_data['address']=union_data['home_address']+union_data['work_address']
union_data['car']=union_data['car']+union_data['car_type']

union_data=union_data.drop(columns=['home_address','work_address','car_type'])

### Нормализация числовых столбцов ()
После нормализации показатель ROC ощутимо вырос - логистическая регрессия чувствительна к разнистя порядков числовых признаков

In [143]:

column_names_to_normalize = ['date_age','income', 'decline_app_cnt']#'date_age']'bki_request_cnt',

X= union_data[column_names_to_normalize].values
MinMaxScaler = StandardScaler()
x_scaled = MinMaxScaler.fit_transform(X)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = union_data.index)
union_data[column_names_to_normalize] = df_temp


### Обучаем модель и получаем первые результаты

In [144]:

# Делим train на тренировочную и валидационную
X_train, X_valid, Y_train, Y_valid=split_train_valid (union_data, 'train', 'test_file', 'default')

# Обучаем модель:

model_1 = LogisticRegression(solver='liblinear')
model_1.fit(X_train, Y_train)
Y_pred = model_1.predict(X_valid)

# Строим ROC AUC 

probs = model_1.predict_proba(X_valid)
probs = probs[:,1]
roc(Y_valid,probs)

# Строим матрицу ошибок

conf_matrix(model_1,X_valid,Y_valid)

# Остальные метрики
metrics(Y_valid, Y_pred)


Нормализация дала небольшое улучшение, модель стала хотя бы давать отказы по кредиту. Однако ложно-отрицательных оценок оказалось на порядок больше, чем истино-отрицательных, Показатели точности и полноты (P и R) недопустимо низкие, комплексная оценка (F-мера) также никуда не годится. Модель невозможно использовать на данном этапе.

Посмотрим еще раз на распределение после преобразований и прологарифмируем числовые признаки:

In [145]:
num_cols
cols_to_ln=['age','decline_app_cnt','income']#'bki_request_cnt',
#Опытным путем выяснено, что логарифмирование bki_request_cnt не улучшает модель, исключаем его из обработки

fig, axes = plt.subplots(1, 3, figsize=(20,5))
print ('До логарифмирования')
for col, i in zip(cols_to_ln, range(3)):
    sns.histplot(union_data[col], kde=False, ax=axes.flat[i])
plt.show()
union_data_ln=union_data.copy()
#(np.log(union_data[cols_to_ln]+1)).hist(figsize=(25,10),bins=100)
for i in cols_to_ln:
    union_data_ln[i] = np.log(union_data[i]+1)

fig, axes = plt.subplots(1, 3, figsize=(20,5))    
print ('После логарифмирования')
for col, i in zip(cols_to_ln, range(3)):
    sns.histplot(union_data_ln[col], kde=False, ax=axes.flat[i])
plt.show()    

Оценим результат: логарифмирование числовых признаков существенно не повлияло на итоговый результат, лишь немного улучшило его. Поскольку минимальные, но улучшения есть, оставим логарифмирование.

In [146]:
# Делим train на тренировочную и валидационную
X_train, X_valid, Y_train, Y_valid=split_train_valid (union_data_ln, 'train', 'test_file', 'default')
#обучаем модель
# Обучаем модель:

model_1_1 = LogisticRegression(solver='liblinear')
model_1_1.fit(X_train, Y_train)
Y_pred = model_1_1.predict(X_valid)

# Строим ROC AUC 

probs = model_1_1.predict_proba(X_valid)
probs = probs[:,1]
roc(Y_valid,probs)

# Строим матрицу ошибок

conf_matrix(model_1_1,X_valid,Y_valid)

# Остальные метрики
metrics(Y_valid, Y_pred)


Чуть улучшили результат, но дисбаланс классов по-прежнему сильно влияет на показатели.
Расширим признаковое пространство: Добавим полиномиальные признаки

In [147]:
union_data.head()
poly_cols=['age','decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income', 'date_age']

In [148]:
#Инициализируем класс, который выполняет преобразование"""
pf = PolynomialFeatures(2)
#Обучаем преобразование на обучающей выборке, применяем его к тестовой"""
poly_data = pf.fit_transform(union_data_ln[poly_cols])[:, len(poly_cols):]
poly_cols = pf.get_feature_names()[len(poly_cols):]
poly_df = pd.DataFrame(poly_data, columns=poly_cols)
union_data_ln_poly = union_data_ln.join(poly_df, how='left')


In [149]:
union_data_ln_poly

In [150]:
impotant_cols(union_data_ln_poly,union_data_ln_poly[union_data_ln_poly['test_file'] == 0].columns, union_data_ln_poly.default)

In [151]:
#Оставляем только наиболее значимые признаки
union_data_ln_poly=union_data_ln_poly[['default','client_id','test_file','train','score_bki','sna', 'first_time','address','decline_app_cnt', 'region_rating', 'bki_request_cnt', 'foreign_passport',
                                       'income','income_reg','car','good_work','date_age', 'education_cod','age','sex', 'x0^2','x2 x3']]
#dummy - переменные
#По матрице корреляции найдены наиболее коррелирующие признаки и удалены из обучающей выборки:
#'x0 x3','income_cat',,'income_cat'
union_data_ln_poly_dum = pd.get_dummies(union_data_ln_poly, columns=['education_cod', 'sna', 'first_time', 'address','sex', 'car', 'good_work', 'foreign_passport','region_rating'], dummy_na=False)

In [152]:
plt.rcParams['figure.figsize'] = (15,10)

matrix = np.triu(union_data_ln_poly_dum[union_data_ln_poly_dum['test_file'] == 0].corr())
sns.heatmap(union_data_ln_poly_dum.corr(), annot=False, mask=matrix, cmap= 'coolwarm')

In [153]:
# Делим train на тренировочную и валидационную
X_train, X_valid, Y_train, Y_valid=split_train_valid (union_data_ln_poly_dum, 'train', 'test_file', 'default')
#'decline_app_cnt',
# Обучаем модель:

Подбор гиперпараметров осуществлялся вручную опытным путем. 

In [154]:
# Обучаем модель: 

C= 1 #Коэффициент регуляризации. При усилении или ослаблении регуляризации показатель качества модели падает на валидационной выборке
class_weight='balanced'#Вариант как сбалансировать обучающую выборку. Ниже рассмотрены альтернативные варианта баланисирования. 

n_jobs=1
model = LogisticRegression(C=C, n_jobs=n_jobs, solver='liblinear', class_weight=class_weight ,penalty='l2',max_iter = 1000)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_valid)

# Строим ROC AUC 

probs = model.predict_proba(X_valid)
probs = probs[:,1]
roc(Y_valid,probs)

# Строим матрицу ошибок

conf_matrix(model,X_valid,Y_valid)

# Остальные метрики
metrics(Y_valid, Y_pred)

In [155]:
union_data_ln_poly_dum

Попробуем сбалансирвоать обучающую выборку (поскольку есть явный (естественный) дисбаланс обучающей выборки по классам в сторону положительных оценок).
2.1 Используем under_sampling.

In [156]:
from imblearn.under_sampling import NearMiss, TomekLinks, RandomUnderSampler
R = RandomUnderSampler()
nm = NearMiss()
X_train_miss, Y_train_miss = R.fit_resample(X_train, Y_train)

print('После применения метода кол-во меток со значением default=1: {}'.format(sum(Y_train_miss == 0)))
print('После применения метода кол-во меток со значением default=0: {}'.format(sum(Y_train_miss == 1)))

In [157]:
# Делим train на тренировочную и валидационную
#X_train, X_valid, Y_train, Y_valid=split_train_valid (union_data, 'train', 'test_file', 'default')
#обучаем модель
# Обучаем модель:

model_2_1 = LogisticRegression(solver='liblinear')
model_2_1.fit(X_train_miss, Y_train_miss)
Y_pred = model_2_1.predict(X_valid)

# Строим ROC AUC 

probs = model_2_1.predict_proba(X_valid)
probs = probs[:,1]
roc(Y_valid,probs)

# Строим матрицу ошибок

conf_matrix(model_2_1,X_valid,Y_valid)

# Остальные метрики
metrics(Y_valid, Y_pred)

После перебора различных вариантов undersamoling из библиотеки imblearn был выбран метод, дваший наиболее высокий рост показателей. Таким методом оказался, как ни странно, метод случайной выборки по классу. Методы, позволяющий отобрать соседей, и использующие информацию о соседях, отарботали намного хуже.

Используем over_sampling:

In [158]:
from imblearn.over_sampling import SMOTE, ADASYN,RandomOverSampler
#oversample = SMOTE()
oversample = RandomOverSampler(sampling_strategy='minority')
X_train_over, Y_train_over = oversample.fit_resample(X_train, Y_train)

#nm = NearMiss()
#X_train_miss, Y_train_miss = nm.fit_resample(X_train, Y_train.ravel())

print('После применения метода кол-во меток со значением default=1: {}'.format(sum(Y_train_over == 0)))
print('После применения метода кол-во меток со значением default=0: {}'.format(sum(Y_train_over == 1)))

# Обучаем модель:

model_2_2 = LogisticRegression(solver='liblinear')
model_2_2.fit(X_train_miss, Y_train_miss)
Y_pred = model_2_2.predict(X_valid)

# Строим ROC AUC 

probs = model_2_2.predict_proba(X_valid)
probs = probs[:,1]
roc(Y_valid,probs)

# Строим матрицу ошибок

conf_matrix(model_2_2,X_valid,Y_valid)

# Остальные метрики
metrics(Y_valid, Y_pred)


после Oversampling также видим резкое смещение ответов в пользу ложно-негативных, при том, что показатели остались практически такими же. 
### Вывод по Oversampling и Undersampling
Эти методы дают приблизительно оиднковый результат (при использовании аналогичной логики выборки для удаления или добавления объема уровновешивающей выборки.
Подбор параметров как и способа уравновешивания зависит от модели и требуемого результата. 
В нашем случае алгоритм подбора соседей показал себя очень плохо, лучше всего отработал метод случайной выборки.
### Вывод по class_weight
Пармаетр LogisticRegression, позволяющий устанавливать веса для уравновешивания неравномерно распределенной выборки показа себя в нашем случае лучше, чем уравновешивание обучающей выборки с помощью Oversampling и Undersampling


## Модель для submission

In [159]:
# Делим train на тренировочную и валдационную (оставляем ВСЮ тренировочную выборку)
X_train=union_data_ln_poly_dum.query('test_file==0').drop (columns=['default','test_file','train','client_id'])
Y_train=union_data_ln_poly_dum.query('test_file==0')['default']


# Обучаем модель:


model_total = LogisticRegression(C=C, n_jobs=n_jobs, solver='liblinear', class_weight=class_weight ,penalty='l2',max_iter = 1000)
model_total.fit(X_train, Y_train)

In [160]:
sub_test_union=union_data_ln_poly_dum.query('test_file==1')
#display(sub_test)
sub_test=sub_test_union.drop(['default','test_file','train', 'client_id'], axis=1)
y_pred_final = model_total.predict_proba(sub_test)[:, 1]
submission = pd.concat([sub_test_union.client_id,pd.Series(y_pred_final,name='default')],axis=1)
display(submission)
submission.to_csv('submission.csv', index=False)